In [160]:
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

### DataSource

In [161]:
class DataSource:
    
    def __init__(self, filename):
        self.df = pd.read_csv(filename)
        
    def data_load_split(self, target=None, ignore=None):
        self.target = target
        self.ignore = ignore
        self.inputs = sorted(set(self.df.columns) - set(self.target) - set(self.ignore))
        
        self.X = self.df[self.inputs]
        self.y = self.df[self.target]
        
        return self.X, self.y
    
    def define_problem(self):
        if self.y.dtypes[0] in ['int64', 'float64'] and self.y.nunique()[0] == 2:
            self.problem = "Binary"
        elif self.y.dtypes[0] in ['object', 'bool']:
            self.problem = "Classification"
        else:
            self.problem = "Regression"
    
        return self.problem
    
    def data_preprocess(self, X, y, problem="Regression"):

        # Data type detection
        numerical_ix = self.X.select_dtypes(include=['int64', 'float64']).columns
        categorical_ix = self.X.select_dtypes(include=['object', 'bool']).columns

        # Data transform
        num_transform = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ])
        cat_transform = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value="Missing")),
            ('oh_encoder', OneHotEncoder(sparse=False))
        ])

        transform_x = ColumnTransformer(transformers=[
            ('num', num_transform, numerical_ix),
            ('cat', cat_transform, categorical_ix)
        ])
        
        if problem == "Regression" or "Binary":
            transform_y = ColumnTransformer(transformers=[
                ('num', Normalizer(), y.columns)
            ])
        else:
            transform_y = ColumnTransformer(transformers=[
                ('cat', cat_transform, y.columns)
            ])
            
        self.trans_X = transform_x.fit_transform(self.X)
        self.trans_y = transform_y.fit_transform(self.y)

        return self.trans_X, self.trans_y
    
    def train_val_split(self, X, y, ratio=0.2, random_state=42):
        return train_test_split(X, y, test_size=ratio, random_state=random_state)

In [162]:
filename="./data/titanic_train.csv"
ds = DataSource(filename)

In [163]:
X, y = ds.data_load_split(target=['Survived'], 
                          ignore=["Name", "Cabin", "Ticket"])

In [164]:
problem = ds.define_problem()

In [165]:
trans_X, trans_y = ds.data_preprocess(X, y, problem=problem)

In [166]:
X_train, X_val, y_train, y_val = ds.train_val_split(trans_X, trans_y, ratio=0.2, random_state=42)

In [230]:
class MLP:
    
    def __init__(self, problem="Regression"):
        self.problem = problem
        
        
    def build_structure(self, max_hidden_layers=1, units=[16]):
        self.structures = []
        self.structures_info = []
        self.max_hidden_layers = max_hidden_layers
        self.units = units
    
        grid = [np.arange(self.max_hidden_layers)+1, self.units]
        for param_tuple in itertools.product(*grid):
            structure_param = {'hidden_layers': param_tuple[0],
                                'units': param_tuple[1]}

            # input layer
            model = keras.Sequential()
            model.add(keras.layers.Dense(16, input_shape=(trans_X.shape[1],)))

            # hidden layer block
            for _ in range(structure_param['hidden_layers']):
                model.add(keras.layers.Dense(structure_param['units'], activation='relu'))

            # output layer
            if problem == 'Regression':
                model.add(keras.layers.Dense(1))
            elif problem == 'Binary':
                model.add(keras.layers.Dense(1, activation='sigmoid'))
            else:
                model.add(keras.layers.Dense(trans_y.shape[1], activation='softmax'))
            self.structures.append(model)
            self.structures_info.append(structure_param)

        return self.structures, self.structures_info
    
    
    def create_optimizer(self, optimizers=['adam'], lrs=[0.01], use_all=False):
        self.created_optimizers = []
        self.optimizers_info = []
        self.optimizers = optimizers
        self.lrs = lrs

        self.optimizer_classes = {'adadelta': keras.optimizers.Adadelta, 'sgd': keras.optimizers.SGD,
                                  'adam': keras.optimizers.Adam, 'adagrad': keras.optimizers.Adagrad,
                                  'adamax': keras.optimizers.Adamax, 'rmsprop': keras.optimizers.RMSprop}

        if use_all:
            self.lrs = [0.001, 0.01, 0.02, 0.1]
            opt_grid = [self.optimizer_classes.keys(), self.lrs]    
        else:
            opt_grid = [self.optimizers, self.lrs]

        for opt_tuple in itertools.product(*opt_grid):
            opt_param = {
                'optimizer_name': opt_tuple[0],
                'lr': opt_tuple[1]
            }

            opt_class = self.optimizer_classes.get(opt_param['optimizer_name'])
            self.created_optimizers.append(opt_class(opt_param['lr']))
            self.optimizers_info.append(opt_param)

        return self.created_optimizers, self.optimizers_info
    
    
    def _compile_model(self):

        # compile network
        if self.problem == "Regression":
            self.loss = keras.losses.MSE
            self.metrics = ['MSE', 'MAE']
        elif self.problem == "Binary":
            self.loss = keras.losses.binary_crossentropy
            self.metrics = ['accuracy']
        else:
            self.loss = keras.losses.categorical_crossentropy
            self.metrics = ['accuracy']

        self.compiled_models = []
        self.compiled_models_info = []

        compile_grid = [zip(self.structures, self.structures_info), zip(self.created_optimizers ,self.optimizers_info)]
        for compile_tuple in itertools.product(*compile_grid):
            compile_param = {'model': compile_tuple[0][0],
                             'optimizer': compile_tuple[1][0]}
            model_info = {'structure_info': compile_tuple[0][1],
                           'optimizer_info': compile_tuple[1][1]}

            model = compile_param['model']
            model.compile(optimizer=compile_param['optimizer'],
                          loss=self.loss,
                          metrics=self.metrics)

            self.compiled_models.append(model)
            self.compiled_models_info.append(model_info)

        return self.compiled_models, self.compiled_models_info
    
    
    def train_models(self, models, X_train, y_train, X_val=None, y_val=None,
                     batch_size=None, epochs=1, verbose=1, callbacks=None,
                     validation_data=None, shuffle=True, steps_per_epoch=None):

        self.models = models
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val

        self.trained_models = []
        for model in models:
            model.fit(x=self.X_train, y=self.y_train,
                      batch_size=batch_size, epochs=epochs, verbose=verbose, callbacks=callbacks,
                      validation_data=validation_data, shuffle=shuffle, steps_per_epoch=steps_per_epoch)
            self.trained_models.append(model)
            
        return self.trained_models

In [231]:
mlp = MLP(problem=problem)

In [232]:
mlp.problem

'Binary'

In [233]:
structures, structures_info = mlp.build_structure(max_hidden_layers=2, units=[16, 32])

In [234]:
structures_info

[{'hidden_layers': 1, 'units': 16},
 {'hidden_layers': 1, 'units': 32},
 {'hidden_layers': 2, 'units': 16},
 {'hidden_layers': 2, 'units': 32}]

In [235]:
created_optimizers, optimizers_info = mlp.create_optimizer(optimizers=['adam'], lrs=[0.1])

In [236]:
optimizers_info

[{'optimizer_name': 'adam', 'lr': 0.1}]

In [237]:
compiled_models, compiled_models_info = mlp._compile_model()

In [238]:
compiled_models_info

[{'structure_info': {'hidden_layers': 1, 'units': 16},
  'optimizer_info': {'optimizer_name': 'adam', 'lr': 0.1}},
 {'structure_info': {'hidden_layers': 1, 'units': 32},
  'optimizer_info': {'optimizer_name': 'adam', 'lr': 0.1}},
 {'structure_info': {'hidden_layers': 2, 'units': 16},
  'optimizer_info': {'optimizer_name': 'adam', 'lr': 0.1}},
 {'structure_info': {'hidden_layers': 2, 'units': 32},
  'optimizer_info': {'optimizer_name': 'adam', 'lr': 0.1}}]

In [239]:
trained_models = mlp.train_models(compiled_models, X_train, y_train, X_val, y_val)

Train on 837 samples
837/837 [==============================] - 0s 306us/sample - loss: 0.5423 - accuracy: 0.7646
Train on 837 samples
837/837 [==============================] - 0s 312us/sample - loss: 0.5819 - accuracy: 0.7384
Train on 837 samples
837/837 [==============================] - 0s 333us/sample - loss: 0.5789 - accuracy: 0.7431
Train on 837 samples
837/837 [==============================] - 0s 324us/sample - loss: 0.6839 - accuracy: 0.6487


In [240]:
trained_models

In [245]:
trained_models[0].weights[0]

<tf.Variable 'dense_182/kernel:0' shape=(780, 16) dtype=float64, numpy=
array([[ 0.50513491,  0.31678673,  0.44185563, ...,  0.15268297,
         0.32069284, -0.49380096],
       [-0.35772899, -0.10142673, -0.44176228, ...,  0.11582109,
         0.0400986 , -0.1295844 ],
       [-0.12226534,  0.34473027,  0.4856819 , ..., -1.14256206,
         0.50789395, -0.8634656 ],
       ...,
       [-0.02861953,  0.0062662 ,  0.06411489, ...,  0.07672749,
         0.05053699,  0.04427339],
       [-0.494762  , -0.45752164, -0.34920057, ..., -0.50000736,
         0.420618  ,  0.3527054 ],
       [ 0.07344273,  0.15911021,  0.04997644, ...,  0.11275034,
        -0.10215235,  0.02165389]])>

In [197]:
model_1 = compiled_models[0]
model_1

In [198]:
mlp.

In [159]:
np.random.seed(42)
tf.random.set_seed(42)
model_1 = compiled_models[0]
model_1
np.random.seed(42)
tf.random.set_seed(42)
model_1.fit(trans_X, trans_y,
            batch_size=64, shuffle=True, epochs=30)

Train on 1047 samples
Epoch 1/30
1047/1047 [==============================] - 0s 84us/sample - loss: 0.6631 - accuracy: 0.6218
Epoch 2/30
1047/1047 [==============================] - 0s 48us/sample - loss: 0.6645 - accuracy: 0.6218
Epoch 3/30
1047/1047 [==============================] - 0s 59us/sample - loss: 0.6634 - accuracy: 0.6218
Epoch 4/30
1047/1047 [==============================] - 0s 56us/sample - loss: 0.6643 - accuracy: 0.6218
Epoch 5/30
1047/1047 [==============================] - 0s 61us/sample - loss: 0.6639 - accuracy: 0.6218
Epoch 6/30
1047/1047 [==============================] - 0s 57us/sample - loss: 0.6640 - accuracy: 0.6218
Epoch 7/30
1047/1047 [==============================] - 0s 61us/sample - loss: 0.6638 - accuracy: 0.6218
Epoch 8/30
1047/1047 [==============================] - 0s 64us/sample - loss: 0.6643 - accuracy: 0.6218
Epoch 9/30
1047/1047 [==============================] - 0s 68us/sample - loss: 0.6646 - accuracy: 0.6218
Epoch 10/30
1047/1047 [==========